# Logging examples

## Creating a log file

In [1]:
import logging

logging.warning('This is a warning message')
logging.critical('This is a critical message')
logging.debug('debug')

CRITICAL:root:This is a critical message


## Logging levels

Level | Description
:-- | :--
``CRITICAL`` | The programme was stopped
``ERROR`` | A serious error has occurred
``WARNING`` | An indication that something unexpected has happened (default level)
``INFO`` | Confirmation that things are working as expected
``DEBUG`` | Detailed information that is usually only of interest when diagnosing problems

### Setting the logging level

In [2]:
import logging

logging.basicConfig(filename="example.log", filemode="w", level=logging.INFO)

logging.info("Informational message")
logging.error("An error has happened!")

ERROR:root:An error has happened!


## Creating a Logger Object

In [3]:
import logging

logging.basicConfig(filename="example.log")
logger = logging.getLogger("example")
logger.setLevel(logging.INFO)
 
try:
    raise RuntimeError
except Exception:
    logger.exception("Error!")

ERROR:example:Error!
Traceback (most recent call last):
  File "/var/folders/f8/0034db6d78s5r6m34fxhpk7m0000gp/T/ipykernel_7239/1794580270.py", line 8, in <module>
    raise RuntimeError
RuntimeError


## Logging exceptions

In [4]:
try:
    1 / 0
except ZeroDivisionError:
    logger.exception("You can’t do that!")

ERROR:example:You can’t do that!
Traceback (most recent call last):
  File "/var/folders/f8/0034db6d78s5r6m34fxhpk7m0000gp/T/ipykernel_7239/760044062.py", line 2, in <module>
    1 / 0
ZeroDivisionError: division by zero


## Logging handler

#### Handler types

Handler | Description
:--- | :---
``StreamHandler`` | ``stdout``, ``stderr`` or file-like objects
``FileHandler`` | for writing to disk
``RotatingFileHandler`` | supports log rotation
``TimedRotatingFileHandler`` | supports the rotation of log files on the hard disk at specific time intervals
``SocketHandler`` | sends logging output to a network socket
``SMTPHandler`` | supports sending logging messages to an e-mail address via SMTP

> **See also:**
> 
> Further handlers can be found at [Logging handlers](https://docs.python.org/3/library/logging.handlers.html#module-logging.handlers)

### StreamHandler

In [5]:
import logging

logger = logging.getLogger('stream_logger')
logger.setLevel(logging.INFO)

console = logging.StreamHandler()

logger.addHandler(console)
logger.info("This is an informational message")

This is an informational message
INFO:stream_logger:This is an informational message


### SMTPHandler

In [6]:
import logging
import logging.handlers

logger = logging.getLogger("email_logger")
logger.setLevel(logging.INFO)
fh = logging.handlers.SMTPHandler('localhost',
                                  fromaddr='python-log@localhost',
                                  toaddrs=['logs@cusy.io'],
                                  subject='Python log')
logger.addHandler(fh)
logger.info("This is an informational message")

Message: 'This is an informational message'
Arguments: ()
INFO:email_logger:This is an informational message


## Log formatting

You can use formatters to format log messages.

In [7]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')

Besides ``%(asctime)s``, ``%(name)s`` and ``%(message)s`` you will find other attributes in [LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes).

In [8]:
import logging

logger = logging.getLogger('stream_logger')
logger.setLevel(logging.INFO)

console = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')
console.setFormatter(formatter)

logger.addHandler(console)
logger.info("This is an informational message")

This is an informational message
2021-12-12 21:20:55,834 - stream_logger - This is an informational message
INFO:stream_logger:This is an informational message


> **Note:**
> 
> The logging module is thread-safe. However, logging may not work in asynchronous contexts. In such cases, however, you can use the [QueueHandler](https://docs.python.org/3/library/logging.handlers.html#queuehandler).

> **See also:**
> 
> [Logging to a single file from multiple processes](https://docs.python.org/3/howto/logging-cookbook.html#logging-to-a-single-file-from-multiple-processes)

## Logging to multiple handlers

In [9]:
import logging

def log(path, multipleLocs=False): 
    logger = logging.getLogger("Example_logger_%s" % fname)
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler(path)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    if multipleLocs:
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        console.setFormatter(formatter)
        logger.addHandler(console)

    logger.info("This is an informational message")
    try:
        1 / 0
    except ZeroDivisionError:
        logger.exception("You can’t do that!")

    logger.critical("This is a no-brainer!")

## Configure logging

> **See also:**
> 
> [logging configuration](https://docs.python.org/3/howto/logging.html#configuring-logging)

### … in an INI file

In the following example, the file `development.ini` is loaded in this directory:

```ini
[loggers]
keys=root

[handlers]
keys=stream_handler

[formatters]
keys=formatter

[logger_root]
level=DEBUG
handlers=stream_handler

[handler_stream_handler]
class=StreamHandler
level=DEBUG
formatter=formatter
args=(sys.stderr,)

[formatter_formatter]
format=%(asctime)s %(name)-12s %(levelname)-8s %(message)s
```

In [11]:
from logging.config import fileConfig
import logging
import logging.config

logging.config.fileConfig('development.ini')
logger = logging.getLogger("example")

logger.info("Program started")
logger.info("Done!")

**Pro:**

* Ability to update the configuration on the fly by using the `logging.config.listen()` function to listen on a socket.
* Different configurations can be used in different environments, so for example, `DEBUG` can be specified as the log level in `development.ini`, while `WARN` is used in `production.ini`.

**Con:**

* Less control for example over custom filters or loggers configured in code.

### … in a dictConfig

In [12]:
import logging
import logging.config

dictLogConfig = {
    "version":1,
    "handlers":{
                "fileHandler":{
                    "class":"logging.FileHandler",
                    "formatter":"exampleFormatter",
                    "filename":"dict_config.log"
                    }
                },
    "loggers":{
        "exampleApp":{
            "handlers":["fileHandler"],
            "level":"INFO",
            }
        },

    "formatters":{
        "exampleFormatter":{
            "format":"%(asctime)s - %(name)s - %(levelname)s - %(message)s"
            }
        }
    }


In [13]:
logging.config.dictConfig(dictLogConfig)

logger = logging.getLogger("exampleApp")

logger.info("Program started")
logger.info("Done!")

2021-12-12 21:22:14,326 exampleApp   INFO     Program started
2021-12-12 21:22:14,329 exampleApp   INFO     Done!


**Pro:**

* Update on the fly

**Con:**

* Less control than configuring a logger in code

### … directly in the code

In [14]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

## *Magic Commands*

| Befehl         | Beschreibung                                                                              |
| -------------- | ----------------------------------------------------------------------------------------- |
| `%logstart`    | Starts logging anywhere in a session                                                      |
|                | `%logstart [-o\|-r\|-t\|-q] [log_name [log_mode]]`                                        |
|                | If no name is given, `ipython_log.py` is used in the current directory.                   |
|                | `log_mode` is an optional parameter. The following modes can be specified:                |
|                | * `append` appends the logging information to the end of an existing file                 |
|                | * `backup` renames the existing file to `name~` and writes to `name`                      |
|                | * `global` appends the logging information at the end of an existing file                 |
|                | * `over` overwrites an existing log file                                                  |
|                | * `rotate` creates rotating log files: `name.1~`, `name.2~`, etc.                         |
|                | Options:                                                                                  |
|                | * `-o` also logs the output of IPython                                                    |
|                | * `-r` logs raw output                                                                    |
|                | * `-t` writes a time stamp in front of each log entry                                     |
|                | * `-q` suppresses the logging output                                                      |
| `%logon`       | Restart the logging                                                                       |
| `%logoff`      | Temporary termination of logging                                                          |

**Pro:**

* Complete control over the configuration

**Con:**

* Changes in the configuration require a change in the source code

## Logs rotate

In [15]:
import logging
import time
from logging.handlers import RotatingFileHandler

def create_rotating_log(path):
    logger = logging.getLogger("Rotating Log")
    logger.setLevel(logging.INFO)

    handler = RotatingFileHandler(path, maxBytes=20,
                                  backupCount=5)
    logger.addHandler(handler)

    for i in range(10):
        logger.info("This is an example log line %s" % i)
        time.sleep(1.5)


if __name__ == "__main__":
    log_file = "rotated.log"
    create_rotating_log(log_file)

2021-12-12 21:22:33,962 Rotating Log INFO     This is an example log line 0
2021-12-12 21:22:33,962 Rotating Log INFO     This is an example log line 0
2021-12-12 21:22:35,471 Rotating Log INFO     This is an example log line 1
2021-12-12 21:22:35,471 Rotating Log INFO     This is an example log line 1


### Rotate logs time-controlled

In [ ]:
import logging
import time
from logging.handlers import TimedRotatingFileHandler


def create_timed_rotating_log(path):
    """"""
    logger = logging.getLogger("Rotating Log")
    logger.setLevel(logging.INFO)

    handler = TimedRotatingFileHandler(path,
                                       when="s",
                                       interval=5,
                                       backupCount=5)
    logger.addHandler(handler)

    for i in range(6):
        logger.info("This is an example!")
        time.sleep(75)

if __name__ == "__main__":
    log_file = "timed_rotation.log"
    create_timed_rotating_log(log_file)

2021-12-12 21:22:58,287 Rotating Log INFO     This is an example!
2021-12-12 21:22:58,287 Rotating Log INFO     This is an example!


## Create a logging decorator

> **See also:**
> 
> [How to Create an Exception Logging Decorator](https://www.blog.pythonlibrary.org/2016/06/09/python-how-to-create-an-exception-logging-decorator/)

## Create a logging filter

In [ ]:
import logging
import sys

class ExampleFilter(logging.Filter):
    def filter(self, record):
        if record.funcName == 'foo':
            return False
        return True

logger = logging.getLogger('filter_example')
logger.addFilter(ExampleFilter())

def foo():
    """
    Ignore this function’s log messages
    """
    logger.debug('Message from function foo')

def bar():
    logger.debug('Message from bar')

if __name__ == '__main__':
    logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)
    foo()
    bar()